# Visualize PoPS Global Forecast Result

Use this notebook to generate maps and plots to visualize the aggregated forecast results. 

This notebook can be run after 0, 1, 2, 3a (optional), 3b, and 3c.

All plots are shown in the notebook, and saved as .PNG to the results folder. 
There are many plot options given here. You can run all of them and select only 
those of interest to use, or selectively run specific plots. 

## Set up workspace from env and configuration files 

First, import needed packages.

In [ ]:
import os
import glob
import dotenv
import json

import pandas as pd
import numpy as np
import geopandas
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable
import math

import warnings
warnings.filterwarnings(action='once')

Navigate to main repository.

In [ ]:
# Navigate one level up to the main repository
os.chdir("..")

Read in path variables from .env.

In [ ]:
# Read environmental variables
env_file = os.path.join(".env")
dotenv.load_dotenv(env_file)

input_dir = os.getenv("INPUT_PATH")
out_dir = os.getenv("OUTPUT_PATH")
countries_path = os.getenv("COUNTRIES_PATH")
sim_name = os.getenv("SIM_NAME")

In [ ]:
# Path to forecast results
run_name = f"{sim_name}_forecast"
results_dir = f"{out_dir}/{run_name}"

Read in parameters from config.json

In [ ]:
# Create the path to the forecasted model outputs
config_json_path = f"{out_dir}/config_{sim_name}.json"

with open(config_json_path) as json_file:
    config = json.load(json_file)

Create directories for additional summary statistics and visuals

In [ ]:
if not os.path.exists(f"{out_dir}/summary_stats/{run_name}/"):
    os.makedirs(f"{out_dir}/summary_stats/{run_name}/")

if not os.path.exists(f"{results_dir}/figs/"):
    os.makedirs(f"{results_dir}/figs/")

In [ ]:
# Native countries list
native_countries_ISO3 = config["native_countries_list"]

# Country of interest
coi_ISO3 = config["coi"]

# Summary statistic years
sim_years = config["sim_years"]

# How many samples were run?
num_runs = len(glob.glob(f"{results_dir}/*/*/origin_destination.csv"))

Open the country file

In [ ]:
# Read country file

countries_geo = geopandas.read_file(countries_path)
countries = countries_geo.iloc[:, [4]]
countries.set_index("NAME")
countries_firstintro = countries.iloc[:, [0]]
countries_reintros = countries.iloc[:, [0]]

# Extract full name of COI
coi = countries_geo.loc[countries_geo["ISO3"] == coi_ISO3, "NAME"].values[0]
native_countries_list = countries_geo.loc[countries_geo["ISO3"].isin(native_countries_ISO3), "NAME"].values


## Import and aggregate model outputs

Open all model outputs from individual runs, and read aggregate first introductions and re-introductions.

Note: Once you've gone through this step once, you can skip ahead and read in the aggregated data from .csv to save time.

In [ ]:
# Only run the first time you use this notebook - this cell may be time consuming
paths = glob.glob(f"{results_dir}/*/*/origin_destination.csv")

org_dest_all = pd.DataFrame()
first_intros_all = pd.DataFrame()
first_exports_all = pd.DataFrame()
country_count = (pd.DataFrame(index=countries.iloc[:, 0], columns=["count"])).fillna(0)
coi_first_intros_by_origin = pd.DataFrame()

for sample, path in enumerate(paths):
    path_in_str = str(path)
    org_dest = (pd.read_csv(path)).iloc[:, 1:4]
    org_dest["TS"] = org_dest["TS"].astype(str)
    org_dest["TS"] = org_dest.TS.str[:4].astype(int)
    org_dest_all = org_dest_all.append(org_dest)

    intros = org_dest.iloc[:, 1:4]
    intros = intros.rename(columns={"Destination": "NAME", "TS": sample})
    firstintro = intros.drop_duplicates(subset=["NAME"])
    countries_firstintro = pd.merge(
        countries_firstintro, firstintro, on="NAME", how="left"
    )
    reintros = intros.groupby("NAME").count()
    countries_reintros = pd.merge(countries_reintros, reintros, on="NAME", how="left")

    # get list of countries in transmission network and add to count
    run_countries = list(set(list(org_dest.Origin) + list(org_dest.Destination)))
    for country in run_countries:
        country_count.loc[country] = country_count.loc[country] + 1

    # identify first introductions to each country
    first_intro = org_dest.drop_duplicates(subset=["Destination"])
    first_intros_all = first_intros_all.append(first_intro, ignore_index=True)

    # identify first export from each country
    first_export = org_dest.drop_duplicates(subset=["Origin"])
    first_exports_all = first_exports_all.append(first_export, ignore_index=True)

    # COI first intros by origin
    coi_first_intros = (org_dest[org_dest["Destination"] == coi]).drop_duplicates(
        subset=["Origin"]
    )
    coi_first_intros_by_origin = coi_first_intros_by_origin.append(
        coi_first_intros, ignore_index=True
    )

# Save all summaries

org_dest_all.to_csv(f"{out_dir}/summary_stats/{run_name}/org_dest_all.csv", index=False)
country_count.reset_index().to_csv(
    f"{out_dir}/summary_stats/{run_name}/country_count.csv", index=False
)
first_intros_all.to_csv(
    f"{out_dir}/summary_stats/{run_name}/first_intros_all.csv", index=False
)
first_exports_all.to_csv(
    f"{out_dir}/summary_stats/{run_name}/first_exports_all.csv", index=False
)
coi_first_intros_by_origin.to_csv(
    f"{out_dir}/summary_stats/{run_name}/coi_first_intros_by_origin.csv", index=False
)
countries_firstintro.reset_index().to_csv(
    f"{out_dir}/summary_stats/{run_name}/countries_firstintro.csv", index=False
)
countries_reintros.reset_index().to_csv(
    f"{out_dir}/summary_stats/{run_name}/countries_reintros.csv", index=False
)

In [ ]:
# If you have already created the summary dataframes, you can restart here: 

org_dest_all = pd.read_csv(
    f"{out_dir}/summary_stats/{run_name}/org_dest_all.csv"
)
country_count = pd.read_csv(
    f"{out_dir}/summary_stats/{run_name}/country_count.csv"
)
first_intros_all = pd.read_csv(
    f"{out_dir}/summary_stats/{run_name}/first_intros_all.csv"
)
first_exports_all = pd.read_csv(
    f"{out_dir}/summary_stats/{run_name}/first_exports_all.csv"
)
coi_first_intros_by_origin = pd.read_csv(
    f"{out_dir}/summary_stats/{run_name}/coi_first_intros_by_origin.csv"
)
countries_firstintro = pd.read_csv(
    f"{out_dir}/summary_stats/{run_name}/countries_firstintro.csv", 
    index_col = 0
)
countries_reintros = pd.read_csv(
    f"{out_dir}/summary_stats/{run_name}/countries_reintros.csv", 
    index_col = 0
)

Set indices for first intros and re-intros to country name. 

In [ ]:
countries_firstintro = countries_firstintro.set_index("NAME")
countries_reintros = countries_reintros.set_index("NAME")

Create a separate dataframe from the perspective of the "country of interest" (coi).

In [ ]:
coi_intros = first_intros_all[first_intros_all["Destination"] == coi]
coi_intros = coi_intros.groupby("Origin").count()[["Destination"]]
coi_intros = coi_intros.rename(columns={"Destination": "COI source"})

countries_geo = countries_geo.merge(
    coi_intros, how="left", left_on="NAME", right_on="Origin"
)

Capture statistical moments of the introduction year distribution: mean, mode, min, max, and range.

In [ ]:
arr_yr_mean_all = []
arr_yr_mode_all = []
arr_yr_min_all = []
arr_yr_max_all = []
arr_yr_range_all = []
intro_proportion_all = []
for row in range(len(countries_firstintro.index)):
    runs_no_intro = countries_firstintro.iloc[row].isnull().sum()
    intro_proportion = 1 - (runs_no_intro / len(countries_firstintro.columns))
    intro_proportion_all.append(intro_proportion)
    if intro_proportion == 0:
        arr_yr_min_all.append(None)
        arr_yr_max_all.append(None)
        arr_yr_mean_all.append(None)
        arr_yr_mode_all.append(None)
        arr_yr_range_all.append(None)

    else:
        arr_yr_min = countries_firstintro.iloc[row].min()
        arr_yr_min_all.append(arr_yr_min)
        arr_yr_max = countries_firstintro.iloc[row].max()
        arr_yr_max_all.append(arr_yr_max)
        arr_yr_mean = math.floor(np.nanmean(countries_firstintro.iloc[row]))
        arr_yr_mean_all.append(arr_yr_mean)
        arr_yr_mode = countries_firstintro.iloc[row].mode()
        if len(arr_yr_mode) > 1:
            arr_yr_mode = int(arr_yr_mode.mean())
        else:
            arr_yr_mode = arr_yr_mode[0]
        arr_yr_mode_all.append(arr_yr_mode)
        arr_yr_range_all.append(arr_yr_max - arr_yr_min)

countries_firstintro["arr_yr_mean"] = arr_yr_mean_all
countries_firstintro["arr_yr_mode"] = arr_yr_mode_all
countries_firstintro["arr_yr_min"] = arr_yr_min_all
countries_firstintro["arr_yr_max"] = arr_yr_max_all
countries_firstintro["arr_yr_range"] = arr_yr_range_all
countries_firstintro["intro_proportion"] = intro_proportion_all
countries_firstintro.loc[native_countries_list, "arr_yr_mean"] = None
countries_firstintro.loc[native_countries_list, "arr_yr_mode"] = None
countries_firstintro.loc[native_countries_list, "arr_yr_min"] = None
countries_firstintro.loc[native_countries_list, "arr_yr_max"] = None
countries_firstintro.loc[native_countries_list, "arr_yr_range"] = None
countries_firstintro.loc[native_countries_list, "intro_proportion"] = None

countries_reintros = countries_reintros.fillna(0)
countries_reintros["num_reintros_mean"] = round(countries_reintros.mean(axis=1)).astype(
    int
)
countries_reintros.at[native_countries_list, "num_reintros_mean"] = None

In [ ]:
countries_geo = countries_geo.merge(countries_firstintro["arr_yr_mean"], on="NAME")
countries_geo["arr_yr_mean"] = countries_geo["arr_yr_mean"].astype("Int64")

countries_geo = countries_geo.merge(countries_firstintro["arr_yr_mode"], on="NAME")
countries_geo["arr_yr_mode"] = countries_geo["arr_yr_mode"].astype("Int64")

countries_geo = countries_geo.merge(countries_firstintro["arr_yr_min"], on="NAME")
countries_geo["arr_yr_min"] = countries_geo["arr_yr_min"].astype("Int64")

countries_geo = countries_geo.merge(countries_firstintro["arr_yr_max"], on="NAME")
countries_geo["arr_yr_max"] = countries_geo["arr_yr_max"].astype("Int64")

countries_geo = countries_geo.merge(countries_firstintro["arr_yr_range"], on="NAME")

countries_geo = countries_geo.merge(countries_firstintro["intro_proportion"], on="NAME")

countries_geo = countries_geo.merge(countries_reintros["num_reintros_mean"], on="NAME")

# Save summary geo to file

countries_geo.to_file(
    f"{out_dir}/summary_stats/{run_name}/country_intros.gpkg", index=False, driver="GPKG"
)

## Option: Read in aggregated outputs
If you've already gone through the steps above, you can start from here and read in the aggregated outputs.

In [ ]:
countries_geo = geopandas.read_file(
    f"{out_dir}/summary_stats/{run_name}/country_intros.gpkg"
)

org_dest_all = pd.read_csv(f"{out_dir}/summary_stats/{run_name}/org_dest_all.csv")
country_count = pd.read_csv(f"{out_dir}/summary_stats/{run_name}/country_count.csv")
first_intros_all = pd.read_csv(
    f"{out_dir}/summary_stats/{run_name}/first_intros_all.csv"
)
first_exports_all = pd.read_csv(
    f"{out_dir}/summary_stats/{run_name}/first_exports_all.csv"
)
coi_first_intros_by_origin = pd.read_csv(
    f"{out_dir}/summary_stats/{run_name}/coi_first_intros_by_origin.csv"
)

## Create plots: maps and other figures

### Maps: 

What was the proportion of runs with introductions, for each country? 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="2%", pad=0.1)
ax.set_title("Proportion of Runs with Introductions\n" + run_name, fontsize=18)
countries_geo.plot(
    column="intro_proportion",
    ax=ax,
    legend=True,
    legend_kwds={"label": "proportion"},
    missing_kwds={"color": "lightgrey"},
    cax=cax,
)
plt.savefig(results_dir + "/figs/intro_proportion.png")
plt.show()

What countries were introduction sources to the Country of Interest? How many total introductions were there from each, across runs?

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="2%", pad=0.1)
ax.set_title("Introduction Sources for " + coi + "\n" + run_name, fontsize=18)
countries_geo.plot(
    column="COI source",
    ax=ax,
    legend=True,
    legend_kwds={"label": "intro source count"},
    missing_kwds={"color": "lightgrey"},
    cax=cax,
)
plt.savefig(results_dir + "/figs/" + coi + "_intro_sources.png")
plt.show()

What was the mean number of re-introductions to each country, cross runs?

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="2%", pad=0.1)
ax.set_title("Number of Reintroductions (mean)\n" + run_name, fontsize=18)
countries_geo.plot(
    column="num_reintros_mean",
    ax=ax,
    legend=True,
    legend_kwds={"label": "reintroductions"},
    missing_kwds={"color": "lightgrey"},
    cax=cax,
)
plt.savefig(results_dir + "/figs/num_reintros.png")
plt.show()

The next several plots present the mean, min, max, and range of years of first introduction to each country:

In [ ]:
countries_geo["arr_yr_mean"] = countries_geo["arr_yr_mean"].astype('Int64')

fig, ax = plt.subplots(1, 1, figsize=(12, 12))
plt.title("Year of First Introduction (mean)\n" + run_name, fontsize=18)
countries_geo.plot(
    column="arr_yr_mean",
    categorical=True,
    cmap="viridis",
    legend=True,
    ax=ax,
    missing_kwds={"color": "lightgrey"},
    legend_kwds={"loc": "lower left"},
)
plt.savefig(results_dir + "/figs/first_intros_mean.png")
plt.show()

In [ ]:
countries_geo["arr_yr_min"] = countries_geo["arr_yr_min"].astype('Int64')

fig, ax = plt.subplots(1, 1, figsize=(12, 12))
plt.title("Year of First Introduction (min)\n" + run_name, fontsize=18)
countries_geo.plot(
    column="arr_yr_min",
    categorical=True,
    cmap="viridis",
    legend=True,
    ax=ax,
    missing_kwds={"color": "lightgrey"},
    legend_kwds={"loc": "lower left"},
)
plt.savefig(results_dir + "/figs/first_intros_min.png")
plt.show()

In [ ]:
countries_geo["arr_yr_max"] = countries_geo["arr_yr_max"].astype('Int64')

fig, ax = plt.subplots(1, 1, figsize=(12, 12))
plt.title("Year of First Introduction (max)\n" + run_name, fontsize=18)
countries_geo.plot(
    column="arr_yr_max",
    categorical=True,
    cmap="viridis",
    legend=True,
    ax=ax,
    missing_kwds={"color": "lightgrey"},
    legend_kwds={"loc": "lower left"},
)

plt.savefig(results_dir + "/figs/first_intros_max.png")
plt.show()

In [ ]:
countries_geo["arr_yr_range"] = countries_geo["arr_yr_range"].astype('Int64')

fig, ax = plt.subplots(1, 1, figsize=(12, 12))
plt.title("Year of First Introduction (range)\n" + run_name, fontsize=18)
countries_geo.plot(
    column="arr_yr_range",
    categorical=True,
    cmap="viridis",
    legend=True,
    ax=ax,
    missing_kwds={"color": "lightgrey"},
    legend_kwds={"loc": "lower left"},
)

plt.savefig(results_dir + "/figs/first_intros_range.png")
plt.show()

The next two plots limit the visualizations to only include countries that received introductions in over 50% of the model runs:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 20))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="2%", pad=0.1)
ax.set_title("Portion of Runs with Introductions (>50%)\n" + run_name, fontsize=18)
countries_geo["intro_proportion"] = np.where(
    countries_geo["intro_proportion"] < 0.5, np.nan, countries_geo["intro_proportion"]
)
countries_geo.plot(
    column="intro_proportion",
    ax=ax,
    legend=True,
    legend_kwds={"label": "proportion"},
    missing_kwds={"color": "lightgrey"},
    cax=cax,
)

plt.savefig(
    results_dir + "/figs/intro_proportion_more50pct.png",
    bbox_inches="tight",
    pad_inches=0.01,
)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 20))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="2%", pad=0.1)
ax.set_title(
    "Mean Year of First Introduction (mean of runs with intros >50%)\n" + run_name,
    fontsize=18,
)
countries_geo["arr_yr_mean"] = np.where(
    countries_geo["intro_proportion"] < 0.5, np.nan, countries_geo["arr_yr_mean"]
)
countries_geo.plot(
    column="arr_yr_mean",
    categorical=True,
    cmap="viridis",
    legend=True,
    ax=ax,
    missing_kwds={"color": "lightgrey"},
    legend_kwds={"loc": "lower left"},
)

plt.savefig(
    results_dir + "/figs/first_intro_mean_more50pct.png",
    bbox_inches="tight",
    pad_inches=0.01,
)
plt.show()

### Additional plots: histograms and heatmap of introductions by country source

Assess bridgehead populations with a temporal heatmap:

In [ ]:
destinations_all = list(set(first_intros_all["Destination"]))
min_intro_prop = 0.5
destinations = []
for i in range(len(destinations_all)):
    if (
        len(
            first_intros_all.loc[first_intros_all["Destination"] == destinations_all[i]]
        )
        > num_runs * min_intro_prop
    ):
        destinations.append(destinations_all[i])
num_destinations = len(destinations)

In [ ]:
# Count origins for each timestep
origin_countries_by_ts = (
    pd.DataFrame(index=countries.iloc[:, 0], columns=sim_years)
).fillna(0)
origins = (org_dest_all.groupby(["Origin", "TS"]).count()).reset_index().fillna(0)
bridgehead_years = range(origins.TS.min(), origins.TS.max()+1)

for i in range(len(origins)):
    origin = origins.iloc[i, :]
    origin_countries_by_ts.loc[origin.Origin, origin.TS] = origin.Destination

In [ ]:
# Create heatmap of bridgehead introductions
origin_countries_by_ts_filtered = origin_countries_by_ts[
    origin_countries_by_ts.sum(axis=1) > 0][bridgehead_years].fillna(0)
fig, ax = plt.subplots(figsize=(12, 8))
plt.subplots_adjust(left=0.22, right=1, top=0.92)
res = sns.heatmap(
    origin_countries_by_ts_filtered.drop(native_countries_list, errors="ignore"),
    cmap=sns.color_palette("light:#31688e", as_cmap=True),
    linewidths=0.30,
    annot=False,
    cbar_kws={"label": f"Total Outgoing Transmissions Over {num_runs} Model Runs"},
)
plt.xticks(rotation = 90)
plt.yticks(rotation = 0)
res.set_xticklabels(res.get_xmajorticklabels(), fontsize=14)
res.set_yticklabels(res.get_ymajorticklabels(), fontsize=14)
ax.figure.axes[-1].yaxis.label.set_size(14)
plt.title("Exports from Bridgehead Populations", fontsize=20, pad=15)
plt.ylabel("")
plt.xlabel("Year", fontsize=14)

plt.savefig(f"{results_dir}/figs/bridgehead_sources.png", dpi=600)
plt.show()

Histograms of introductions by year, for individual countries:

In [ ]:
# Plot histograms of first intros by destination
fig, axs = plt.subplots(
    2, math.ceil(num_destinations / 2), sharey=True, sharex=True, figsize=(12, 6)
)
fig.subplots_adjust(hspace=0.35, wspace=0.15, top=0.82)
fig.text(0.5, 0.04, "year", ha="center", fontsize=16)
fig.text(0.08, 0.5, "model runs", va="center", rotation="vertical", fontsize=18)
axs = axs.ravel()

if num_destinations % 2 != 0:
    axs[-1].axis('off')

for i in range(num_destinations):
    axs[i].hist(
        list(
            first_intros_all.loc[
                first_intros_all["Destination"] == destinations[i], "TS"
            ]
        )
    )
    axs[i].set_title(destinations[i])
plt.suptitle(
    f"""{run_name} \n Year of First Introduction by Destination""", fontsize=18
)

plt.savefig(f"{results_dir}/figs/first_intro_by_destination.png")
plt.show()

Write plots for each country to file only:

In [ ]:
# Save separate plots for histograms of first intros by destination
for i in range(num_destinations):
    fig, ax = plt.subplots(1, figsize=(4, 3))
    fig.subplots_adjust(left=0.27, top=0.76, bottom=0.21)
    ax.hist(
        list(
            first_intros_all.loc[
                first_intros_all["Destination"] == destinations[i], "TS"
            ]
        ),
        color="#31688e",
    )
    ax.set_title(f"""{destinations[i]}\nFirst Introduction Year""", fontsize=18, pad=14)
    ax.set_xlabel("year", fontsize=18)
    ax.set_ylabel("% model runs", fontsize=18)
    ax.set_xlim(left=2005, right=2030)
    ax.set_ylim(top=1000)
    y_vals = ax.get_yticks()
    ax.set_xticklabels(["", 2010, "", 2020, "", 2030], fontsize=16)
    ax.set_yticklabels(
        ["{:3.0f}%".format((x / 1000) * 100) for x in y_vals], fontsize=16
    )
    
    plt.savefig(f"{results_dir}/figs/{destinations[i]}_first_intros.png")
    plt.close()

All introductions by country:

In [ ]:
destinations_all = list(set(org_dest_all["Destination"]))
min_intro_prop = 0.5
destinations = []
for i in range(len(destinations_all)):
    if (
        len(org_dest_all.loc[org_dest_all["Destination"] == destinations_all[i]])
        > num_runs * min_intro_prop
    ):
        destinations.append(destinations_all[i])
num_destinations = len(destinations)

In [ ]:
# Plot histograms of all intros by destination
fig, axs = plt.subplots(
    2, math.ceil(num_destinations / 2), sharey=True, sharex=True, figsize=(12, 6)
)
fig.subplots_adjust(hspace=0.35, wspace=0.15, top=0.82)
fig.text(0.5, 0.04, "year", ha="center", fontsize=13)
fig.text(0.08, 0.5, "model runs", va="center", rotation="vertical", fontsize=18)
axs = axs.ravel()

if num_destinations % 2 != 0:
    axs[-1].axis('off')

for i in range(num_destinations):
    axs[i].hist(
        list(org_dest_all.loc[org_dest_all["Destination"] == destinations[i], "TS"])
    )
    axs[i].set_title(destinations[i])
plt.suptitle(f"""{run_name} \n Introductions by Destination""", fontsize=18)
plt.savefig(f"{results_dir}/figs/all_intros_by_destination.png")
plt.show()

Save individual histograms to file:

In [ ]:
# Save separate plots for histograms of all intros by destination
for i in range(num_destinations):
    fig, ax = plt.subplots(1, figsize=(4, 3))
    fig.subplots_adjust(left=0.22, top=0.78, bottom=0.2)
    ax.hist(
        list(org_dest_all.loc[org_dest_all["Destination"] == destinations[i], "TS"]),
        color="#31688e",
    )
    ax.set_title(f"""{destinations[i]}\nIntroduction Year""", fontsize=18, pad=14)
    ax.set_xlabel("year", fontsize=18)
    ax.set_ylabel("% model runs", fontsize=18)
    ax.set_xlim(left=2005, right=2030)
    ax.set_ylim(top=1000)
    y_vals = ax.get_yticks()
    ax.set_xticklabels(["", 2010, "", 2020, "", 2030], fontsize=16)
    ax.set_yticklabels(
        ["{:3.0f}%".format((x / 1000) * 100) for x in y_vals], fontsize=16
    )
    plt.savefig(f"{results_dir}/figs//{destinations[i]}_all_intros.png")
    plt.close()

Introduction summaries for the country of interest:

In [ ]:
# 10th Percentile
def q10(x):
    return x.quantile(0.1)


coi_all_intros = org_dest_all[org_dest_all["Destination"] == coi]

# Save COI intros summaries
coi_all_intros_by_origin_summary = coi_all_intros.groupby(["Origin"]).agg(
    {"TS": ["count", q10, "min", "mean", "median", "max", "std"]}
)
coi_all_intros_by_origin_summary.to_csv(
    f"{out_dir}/summary_stats/{run_name}/all_intros_by_source_to_{coi}.csv"
)
coi_first_intros_by_origin_summary = coi_first_intros_by_origin.groupby(["Origin"]).agg(
    {"TS": ["count", "min", q10, "mean", "median", "max", "std"]}
)
coi_first_intros_by_origin_summary.to_csv(
    f"{out_dir}/summary_stats/{run_name}/first_intro_by_source_to_{coi}.csv"
)

Save individual country histograms to file:

In [ ]:
# Save separate plots for histograms of all COI first intros by origin
coi_origins = list(set(coi_first_intros_by_origin.Origin))
for i in range(len(coi_origins)):
    fig, ax = plt.subplots(1, figsize=(4, 3))
    fig.subplots_adjust(left=0.25, top=0.75, bottom=0.21, right=0.85)
    ax.hist(
        list(
            coi_first_intros_by_origin.loc[
                coi_first_intros_by_origin["Origin"] == coi_origins[i], "TS"
            ]
        ),
        color="#31688e",
    )
    q10_value = q10(
        coi_first_intros_by_origin.loc[
            coi_first_intros_by_origin["Origin"] == coi_origins[i], "TS"
        ]
    )
    ax.axvline(q10_value, color="red")
    plt.text(q10_value - 2.2, 17.5, round(q10_value), rotation=90, fontsize=15)
    ax.set_title(f"""{coi_origins[i]}\nFirst Export to {coi}""", fontsize=18, pad=18)
    ax.set_xlabel("year", fontsize=18)
    ax.set_ylabel("% model runs", fontsize=18)
    ax.set_xlim(left=2005, right=2030)
    ax.set_ylim(top=25)
    y_vals = ax.get_yticks()
    ax.set_xticklabels(["", 2010, "", 2020, "", 2030], fontsize=16)
    ax.set_yticklabels(
        ["{:3.1f}%".format((x / 1000) * 100) for x in y_vals], fontsize=16
    )
    plt.savefig(
        f"{results_dir}/figs/{coi_origins[i]}_first_intros_to_{coi}.png", dpi=300
    )
    plt.close()